In [1]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import time
import mysql.connector
from datetime import datetime # Ini yang tadi kurang
from bs4 import BeautifulSoup

# Library untuk Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def jalankan_proses_scraping():
    print(f"\n🌐 [{datetime.now().strftime('%H:%M:%S')}] Membuka Browser (Stealth Mode)...")
    
    # 1. Konfigurasi Database (Pastikan sesuai dengan XAMPP kamu)
    db_config = {
        'host': '127.0.0.1',
        'user': 'root',
        'password': '',
        'database': 'averroes_sscraping'
    }

    # 2. Konfigurasi Chrome Stealth
    chrome_options = Options()
    chrome_options.add_argument("--headless") # Tanpa jendela (hemat RAM)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    # Inisialisasi Driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        url = "https://cryptowave.co.id/"
        driver.get(url)
        
        # Kasih waktu 5 detik buat render JavaScript & Gambar
        print("⏳ Menunggu konten dimuat sempurna...")
        time.sleep(5) 
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = []
        
        # Mencari artikel (Targeting h1, h2, h3)
        items = soup.select('article, .post-item, .archive-card, .flex.flex-col')

        for item in items:
            title_tag = item.select_one('h1, h2, h3')
            a_tag = title_tag.find('a') if title_tag else item.find('a')
            
            if a_tag and a_tag.get('href') and len(a_tag.text.strip()) > 20:
                judul = a_tag.get_text().strip()
                link = a_tag['href']
                if link.startswith('/'): link = "https://coinvestasi.com" + link
                
                # Cari Gambar yang sudah dirender
                img_tag = item.find('img')
                thumb = None
                if img_tag:
                    thumb = img_tag.get('src')
                
                # Pastikan ini link berita (menghindari menu navigasi)
                if "coinvestasi.com" in link:
                    articles.append((judul, link, thumb))
        
        if articles:
            # SINKRONISASI KE DATABASE
            conn = mysql.connector.connect(**db_config)
            cursor = conn.cursor()
            
            # Hapus data basi (Human-Designed: Menjaga relevansi data)
            cursor.execute("DELETE FROM berita_crypto WHERE waktu_scraped < NOW() - INTERVAL 1 DAY")
            
            # Masukkan data baru (REPLACE agar thumbnail terupdate)
            query = "REPLACE INTO berita_crypto (judul, url, thumbnail) VALUES (%s, %s, %s)"
            unique_articles = list(set(articles)) # Hapus duplikat
            cursor.executemany(query, unique_articles[:15])
            
            conn.commit()
            print(f"✨ Berhasil! {cursor.rowcount} berita & thumbnail masuk ke MySQL.")
            cursor.close()
            conn.close()
        else:
            print("⚠️ Halaman terbuka tapi tidak ada artikel yang terambil.")
        
    except Exception as e:
        print(f"❌ Error saat proses: {e}")
    finally:
        driver.quit() # Penting! Tutup browser agar laptop tidak berat

# Eksekusi
jalankan_proses_scraping()


🌐 [09:46:05] Membuka Browser (Stealth Mode)...
⏳ Menunggu konten dimuat sempurna...
⚠️ Halaman terbuka tapi tidak ada artikel yang terambil.
